In [1]:
import geopandas as gpd
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
import folium
import pandas as pd
import tqdm
import requests
import numpy as np

In [3]:
plt.rcParams['font.family'] = 'Malgun Gothic'
plt.rcParams['figure.figsize'] = (10,10)

## 상권별 유동인구

In [5]:
pop = pd.read_csv('population.csv', encoding = 'cp949')
pop.head()

,기준 년코드,기준_분기_코드,상권_구분_코드,상권_구분_코드_명,상권_코드,상권_코드_명,총_유동인구_수,남성_유동인구_수,여성_유동인구_수,연령대_10_유동인구_수,...,여성연령대_60_이상_토요일시간대_3_유동인구_수,여성연령대_60_이상_토요일시간대_4_유동인구_수,여성연령대_60_이상_토요일시간대_5_유동인구_수,여성연령대_60_이상_토요일시간대_6_유동인구_수,여성연령대_60_이상_일요일시간대_1_유동인구_수,여성연령대_60_이상_일요일시간대_2_유동인구_수,여성연령대_60_이상_일요일시간대_3_유동인구_수,여성연령대_60_이상_일요일시간대_4_유동인구_수,여성연령대_60_이상_일요일시간대_5_유동인구_수,여성연령대_60_이상_일요일시간대_6_유동인구_수
0,2019,3,U,관광특구,1001496,강남 마이스 관광특구,142615,73065,69550,4004,...,169,185,163,39,48,98,156,152,117,29
1,2019,3,U,관광특구,1001495,잠실 관광특구,3259365,1728808,1530557,128520,...,4352,4276,5016,2199,2854,3451,4277,3720,4410,1860
2,2019,3,U,관광특구,1001494,종로?청계 관광특구,7584636,4128195,3456437,224530,...,14675,16319,11950,2976,2549,5068,9379,11357,8151,2131
3,2019,3,U,관광특구,1001493,동대문패션타운 관광특구,5032819,2504148,2528668,131479,...,8251,7827,5504,2359,3293,4588,5205,5412,5226,4002
4,2019,3,U,관광특구,1001492,명동 남대문 북창동 다동 무교동 관광특구,8361221,4087208,4274015,178509,...,21775,23651,12757,1617,1125,5248,11571,11869,7849,1034


In [6]:
pop = pop[pop['기준 년코드'] == 2019]


In [7]:
pop['상권_코드_명'].unique()

array(['강남 마이스 관광특구', '잠실 관광특구', '종로?청계 관광특구', ..., '돈화문로11가길', '난계로27길',
       '계동길'], dtype=object)

In [8]:
columns = ['상권_코드','상권_코드_명','총_유동인구_수']
pop_s = pop[columns]
pop_s.columns = ['code','name','populations']
pop_s.head()

,code,name,populations
0,1001496,강남 마이스 관광특구,142615
1,1001495,잠실 관광특구,3259365
2,1001494,종로?청계 관광특구,7584636
3,1001493,동대문패션타운 관광특구,5032819
4,1001492,명동 남대문 북창동 다동 무교동 관광특구,8361221


In [9]:
pop_s = pd.pivot_table(pop_s, index = 'code', values = 'populations', aggfunc = 'sum').reset_index()

In [10]:
pop_s[pop_s['code'] == 1000001]

,code,populations
0,1000001,536785


## 상권별 위치

In [11]:
loca = pd.read_csv('location.csv', encoding = 'cp949')
loca.head()

,기준_년월_코드,상권_구분_코드,상권_구분_코드_명,상권_코드,상권_코드_명,엑스좌표_값,와이좌표_값,시군구_코드,행정동_코드,형태정보
0,201810,D,발달상권,1001096,신도림역,190279,445432,11530,11530510,NaN
1,201810,R,전통시장,1001453,낙성대시장,196121,442084,11620,11620585,NaN
2,201810,R,전통시장,1001454,봉천제일종합시장,195147,442413,11620,11620595,NaN
3,201810,R,전통시장,1001474,도곡시장,204551,444227,11680,11680650,NaN
4,201810,R,전통시장,1001475,강남개포시장,206065,443310,11680,11680670,NaN


In [12]:
mapo = loca[loca['시군구_코드'] == 11440]
mapo.head()

,기준_년월_코드,상권_구분_코드,상권_구분_코드_명,상권_코드,상권_코드_명,엑스좌표_값,와이좌표_값,시군구_코드,행정동_코드,형태정보
180,201810,A,골목상권,1000505,토정로37길,194987,449132,11440,11440590,NaN
181,201810,A,골목상권,1000506,효창원로93길,196541,449810,11440,11440565,NaN
205,201810,A,골목상권,1000471,마포대로12길,195997,449761,11440,11440555,NaN
206,201810,A,골목상권,1000472,마포대로20길,196235,450199,11440,11440565,NaN
251,201810,A,골목상권,1000467,동교로19길,192667,450953,11440,11440660,NaN


In [13]:
col_m = ['상권_코드','상권_코드_명','엑스좌표_값','와이좌표_값','시군구_코드']
mapo_s = mapo[col_m]
mapo_s.columns = ['code','name','X','Y','code_c']
mapo_s.head()

,code,name,X,Y,code_c
180,1000505,토정로37길,194987,449132,11440
181,1000506,효창원로93길,196541,449810,11440
205,1000471,마포대로12길,195997,449761,11440
206,1000472,마포대로20길,196235,450199,11440
251,1000467,동교로19길,192667,450953,11440


In [14]:
data = pd.merge( pop_s,mapo_s, on = 'code')
data.head()

,code,populations,name,X,Y,code_c
0,1000458,1164943,광성로6길,194578,449861,11440
1,1000459,1482185,대흥로21길,194970,450521,11440
2,1000460,1115615,대흥로28길,195318,450623,11440
3,1000461,703465,도화4길,195659,448858,11440
4,1000462,1726932,도화길,195575,449041,11440


In [15]:
data['name'].unique()

array(['광성로6길', '대흥로21길', '대흥로28길', '도화4길', '도화길', '독막로22길', '독막로32길',
       '독막로8길', '동교로17길', '동교로19길', '동교로25길', '동교로27길', '동교로38길',
       '마포대로12길', '마포대로20길', '마포대로4길', '만리재옛길', '망원로11길', '망원로2길',
       '매봉산로2길', '모래내로7길', '방울내로9안길', '백범로16길', '성미산로11길', '성미산로1길',
       '성미산로32길', '성산로4길', '성지3길', '숭문8길', '숭문길', '신촌로12길', '신촌로28가길',
       '신촌로30길', '양화로11길', '양화로1길', '양화진길', '와우산로29가길', '와우산로29길',
       '와우산로3길', '월드컵로10길', '월드컵로11길', '월드컵로16길', '월드컵로19길', '월드컵로25길',
       '월드컵로36길', '월드컵로7길', '월드컵로8길', '토정로37길', '효창원로93길', '서울 마포구 마포역_1',
       '서울 마포구 마포역_2', '서울 마포구 공덕역_1', '공덕역', '서울 마포구 공덕역_2',
       '서울 마포구 홍대입구역_1', '신촌역', '홍익대학교 주변', '서울 마포구 홍대입구역_2',
       '서울 마포구 홍대입구역_3', '디지털미디어시티', '공덕시장', '마포시장', '아현시장', '신수시장',
       '서교시장', '영진시장', '합정시장', '망원시장', '망원월드컵시장', '동진시장', '마포농수산물시장'],
      dtype=object)

# 데이터 전처리

## 지하철역 근처 상권에 대한 전처리

In [16]:
for i in range(len(data)):
    data['name'][i] = data['name'][i].replace('_1','').replace('_2','').replace('_3','').replace('서울 마포구 ','')


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


## Geocoding을 위한 전처리 작업

In [17]:
for i in range(len(data)):
    data['name'][i] = '서울시 ' + data['name'][i]

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


## 홍익대학교 앞 -> 홍익대학교 바로 앞의 골목 명으로 변경

In [18]:
data['name'][56] = '홍익로'

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


### X, Y좌표 -> 위도 경도로 변환작업

In [19]:
data['y'] = np.nan
data['x'] = np.nan

In [20]:
for i in tqdm.trange(len(data)):
    n = data['name'][i]
    add = 'http://api.vworld.kr/req/address?service=address&request=getcoord&version=2.0&crs=epsg:4326&address=' + n + '&refine=true&simple=false&format=json&type=road&key=DAEEBC94-6866-34E2-BEC6-27D059804F75'
    r = requests.get(add)
    #print(float(r.json()['response']['result']['point']['x']),r.json()['response']['result']['point']['y'])
    data['y'][i] = float(r.json()['response']['result']['point']['x'])
    data['x'][i] = float(r.json()['response']['result']['point']['y'])

  0%|                                                                                           | 0/71 [00:00<?, ?it/s]C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
 85%|█████████████████████████████████████████████████████████████████████▎            | 60/71 [00:09<00:01,  6.58it/s]


KeyError: 'result'

#### 시장의 경우 자동 변환이 불가하여 직접 좌표 입력

In [21]:
data1 = data[['code','populations','name','x','y','X','Y']]
data1.head()

,code,populations,name,x,y,X,Y
0,1000458,1164943,서울시 광성로6길,NaN,NaN,194578,449861
1,1000459,1482185,서울시 대흥로21길,NaN,NaN,194970,450521
2,1000460,1115615,서울시 대흥로28길,NaN,NaN,195318,450623
3,1000461,703465,서울시 도화4길,NaN,NaN,195659,448858
4,1000462,1726932,서울시 도화길,NaN,NaN,195575,449041


### 전처리 및 Geocoding 완료된 자료 저장

In [22]:
#data1.to_csv('market_area.csv', encoding = 'cp949')

In [73]:
data = pd.read_csv('market_area.csv', encoding = 'cp949')

#### 전처리한 wifi 데이터 사용

In [23]:
wifi = pd.read_csv('wifi.csv', encoding = 'cp949')

In [24]:
wifi = wifi[wifi['구명'] == '마포구']

In [ ]:
#wifi.to_csv('wifi.csv', encoding = cp949)

In [25]:
wifi.head()

,구명,유형,설치주소,지역명,상세설치장소,설치기관(회사),설치위치(X좌표),설치위치(Y좌표),콘텐츠아이디
5820,마포구,공원,난지도캠핑장,서울특별시 마포구 한강난지로 28,한강야생탐사센터 옥상,LGU+,126.872436,37.5701575,WF111160
5821,마포구,공원,난지도캠핑장,서울특별시 마포구 한강난지로 28,한강야생탐사센터 옥상,LGU+,126.872436,37.5701575,WF111161
5822,마포구,공원,난지도캠핑장,서울특별시 마포구 한강난지로 28,한강야생탐사센터 옥상,LGU+,126.872436,37.5701575,WF111162
5823,마포구,주요거리,홍대주변#1-1,서울특별시 마포구 양화로 140,홍대입구역 사거리 CCTV,LGU+,126.921698,37.5550793,WF111163
5824,마포구,주요거리,홍대주변#1-2,서울특별시 마포구 양화로 140,홍대입구역 사거리 CCTV,LGU+,126.921698,37.5550793,WF111164


## Blue: 상권, Red: Wifi

In [26]:
map = folium.Map(location=[data['x'].mean(), data['y'].mean()], zoom_start=12, tiles='OpenStreetMap')
for n in data.index:
    popup_name = data['name'][n]

    folium.CircleMarker(
        location=[data['x'][n], data['y'][n]],
        radius=5,
        popup=popup_name,
        fill=True,
        color='#3186cc'
    ).add_to(map)
for n in wifi.index:
    popup_name = wifi['구명'][n]

    folium.CircleMarker(
        location=[wifi['설치위치(Y좌표)'][n], wifi['설치위치(X좌표)'][n]],
        radius=5,
        popup=popup_name,
        fill=True,
        color='red'
    ).add_to(map)
map

In [27]:
import folium

In [28]:
import json
geo_path = 'skorea_municipalities_geo_simple.json'
geo_str = json.load(open(geo_path, encoding='utf-8'))

In [29]:
data['populations'].max()

24909786

In [74]:
data.head()

,Unnamed: 0,code,populations,name,x,y,X,Y
0,0,1000458,1164943,광성로6길,37.549513,126.936671,194578,449861
1,1,1000459,1482185,대흥로21길,37.545634,126.938695,194970,450521
2,2,1000460,1115615,대흥로28길,37.547106,126.941870,195318,450623
3,3,1000461,703465,도화4길,37.537972,126.950394,195659,448858
4,4,1000462,1726932,도화길,37.541531,126.949875,195575,449041


In [75]:
data[['populations','name']].sort_values(by = 'populations', ascending = False).head()

,populations,name
56,24909786,홍익대학교 주변
55,6856474,신촌역
50,4662505,서울 마포구 마포역_2
49,3974972,서울 마포구 마포역_1
58,3004306,서울 마포구 홍대입구역_3


## Wifi 내부 및 외부 설치여부 구분

In [32]:
wifi['유형'].unique()

array(['공원', '주요거리', '공공기관', '문화관광', '전통시장', '복지센터등', '버스승차대'],
      dtype=object)

In [33]:
wifi_out = wifi[wifi['유형']!='공공기관']
wifi_out = wifi_out[wifi_out['유형']!='복지센터등']
wifi_out = wifi_out[wifi_out['유형']!='문화관광']

### Heatmap: 상권별 유동인구, Red: Inside Wifi, Green: Outside Wifi

In [ ]:
max_amount = float(data['populations'].max())

hmap = folium.Map(location=[data['x'].mean(), data['y'].mean()], zoom_start=13.5 )

hm_wide = HeatMap( list(zip(data['x'], data['y'], data['populations'])),
                   min_opacity=0.5,
                   max_val=max_amount,
                   radius=17, blur=15, 
                   max_zoom=1, 
                 )

hm_wide.add_to(hmap)
for n in wifi.index:
    popup_name = wifi['설치주소'][n]

    folium.CircleMarker(
        location=[wifi['설치위치(Y좌표)'][n], wifi['설치위치(X좌표)'][n]],
        radius=5,
        popup=popup_name,
        fill=True,
        color='red'
    ).add_to(hmap)
for n in wifi_out.index:
    popup_name = wifi_out['설치주소'][n]

    folium.CircleMarker(
        location=[wifi_out['설치위치(Y좌표)'][n], wifi_out['설치위치(X좌표)'][n]],
        radius=5,
        popup=popup_name,
        fill=True,
        color='green'
    ).add_to(hmap)

hmap

## 동별 생활인구

In [39]:
s_pop = pd.read_csv('LOCAL_PEOPLE_DONG_201912.csv', index_col = None).reset_index()

In [40]:
s_pop.head()

,index,기준일ID,시간대구분,행정동코드,총생활인구수,남자0세부터9세생활인구수,남자10세부터14세생활인구수,남자15세부터19세생활인구수,남자20세부터24세생활인구수,남자25세부터29세생활인구수,...,여자25세부터29세생활인구수,여자30세부터34세생활인구수,여자35세부터39세생활인구수,여자40세부터44세생활인구수,여자45세부터49세생활인구수,여자50세부터54세생활인구수,여자55세부터59세생활인구수,여자60세부터64세생활인구수,여자65세부터69세생활인구수,여자70세이상생활인구수
0,20191201,0,11215710,45694.9988,452.3937,208.7971,1267.7678,5642.9235,5433.6801,2494.5993,...,2036.1446,1285.4200,794.6241,833.7179,772.1417,773.2152,827.1455,565.5378,1405.3707,NaN
1,20191201,0,11680545,37375.4506,1298.5585,731.2901,889.5126,895.5983,1397.2099,1609.7025,...,1503.9461,1752.1118,1591.0488,1555.4675,1033.6247,1094.5083,988.0638,944.5295,2710.1619,NaN
2,20191201,0,11680610,37266.9526,1083.2150,1559.2566,2339.8178,991.4434,990.1854,946.7321,...,888.9359,997.1139,1738.2101,2423.0246,1408.2536,1184.3355,1071.2366,858.9868,1878.8251,NaN
3,20191201,0,11680655,25231.2064,1191.0306,607.5946,639.2233,608.4116,697.5009,663.3373,...,876.7431,1119.2400,1051.8991,1274.9860,852.3075,933.6367,949.1604,637.7794,1557.6550,NaN
4,20191201,0,11710570,40728.5564,1586.6273,818.9047,1204.9034,1172.2571,1195.7198,1163.9396,...,1321.7052,1625.5710,1434.9812,1861.6645,1766.8905,1991.8166,1977.4260,1332.3513,2779.2676,NaN


In [41]:
s_pop_ex = s_pop[['index','시간대구분','행정동코드']]
colnames = ['day','code','populations']
s_pop_ex.columns = colnames

In [42]:
s_pop_ex.head()

,day,code,populations
0,20191201,11215710,45694.9988
1,20191201,11680545,37375.4506
2,20191201,11680610,37266.9526
3,20191201,11680655,25231.2064
4,20191201,11710570,40728.5564


In [76]:
pop_table = pd.pivot_table(data = s_pop_ex, index = 'code', values = 'populations', aggfunc = 'sum').reset_index()

In [77]:
pop_table.head()

,code,populations
0,11110515,1.229997e+07
1,11110530,2.118485e+07
2,11110540,4.767022e+06
3,11110550,1.191871e+07
4,11110560,1.356408e+07


#### 구를 나타내는 앞 5자리, 동을 나타내는 뒷 3자리 분리

In [78]:
pop_table['code_c'] = pop_table['code'] // 1000

In [79]:
pop_table['code_d'] = pop_table['code'] % 1000

##### 마포구 데이터 추출

In [80]:
mapo_pop = pop_table[pop_table['code_c'] == 11440]

In [81]:
mapo_pop.sort_values(by = 'code_d').head()

,code,populations,code_c,code_d
206,11440555,2.042815e+07,11440,555
207,11440565,2.929401e+07,11440,565
208,11440585,1.674282e+07,11440,585
209,11440590,2.137643e+07,11440,590
210,11440600,1.910087e+07,11440,600


## 동별 코드 매핑

In [49]:
code = pd.read_csv('code.csv',encoding = 'cp949')

In [50]:
code = code.sort_values(by = '시군구코드')

In [51]:
code.head()

,시군구코드,법정동코드,행정동코드,시도명,시군구명,법정동명,행정동명,적용시작일,적용만료일
54313,11000,0,0,서울특별시,NaN,서울특별시,NaN,19880423,99991231
54055,11110,16700,630,서울특별시,종로구,충신동,종로5.6가동,19880423,99991231
54020,11110,18100,580,서울특별시,종로구,행촌동,교남동,19880423,99991231
54021,11110,14600,600,서울특별시,종로구,가회동,가회동,19880423,99991231
54022,11110,14700,600,서울특별시,종로구,재동,가회동,19880423,99991231


In [52]:
mapo_code = code[code['시군구명'] == '마포구']

In [53]:
mapo_code['code'] =mapo_code['법정동코드']//100

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


#### 동별 코드 및 인구 매핑

In [82]:
data = pd.merge( mapo_pop,mapo_code, left_on = 'code_d', right_on = '행정동코드')

In [83]:
data.head()

,code_x,populations,code_c,code_d,시군구코드,법정동코드,행정동코드,시도명,시군구명,법정동명,행정동명,적용시작일,적용만료일,code_y
0,11440555,2.042815e+07,11440,555,11440,10200,555,서울특별시,마포구,공덕동,아현동,20080107,99991231,102
1,11440555,2.042815e+07,11440,555,11440,10900,555,서울특별시,마포구,염리동,아현동,20080107,99991231,109
2,11440555,2.042815e+07,11440,555,11440,10100,555,서울특별시,마포구,아현동,아현동,20080107,99991231,101
3,11440565,2.929401e+07,11440,565,11440,10300,565,서울특별시,마포구,신공덕동,공덕동,20080107,99991231,103
4,11440565,2.929401e+07,11440,565,11440,10100,565,서울특별시,마포구,아현동,공덕동,20080107,99991231,101


In [56]:
print(data['populations'].min(),data['populations'].max())

9313344.144899996 62110058.62770009


In [84]:
data['x'] = np.nan
data['y'] = np.nan

In [85]:
for i in range(len(data)):
    data['행정동명'][i] = data['행정동명'][i].replace('제','')

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [86]:
data['name'] = np.nan

In [87]:
for i in range(len(data)):
    data['name'][i] = data['시군구명'][i]+ ' ' + data['행정동명'][i]

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


#### 각 위치에 대한 위도 경도 Geocoding

In [88]:
for i in tqdm.trange(len(data)):
    n = data['name'][i]
    add = 'http://api.vworld.kr/req/address?service=address&request=getcoord&version=2.0&crs=epsg:4326&address=' + n + '&refine=true&simple=false&format=json&type=road&key=DAEEBC94-6866-34E2-BEC6-27D059804F75'
    r = requests.get(add)
    #print(float(r.json()['response']['result']['point']['x']),r.json()['response']['result']['point']['y'])
    data['y'][i] = float(r.json()['response']['result']['point']['x'])
    data['x'][i] = float(r.json()['response']['result']['point']['y'])

  0%|                                                                                           | 0/43 [00:00<?, ?it/s]C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
100%|██████████████████████████████████████████████████████████████████████████████████| 43/43 [00:10<00:00,  4.21it/s]


In [119]:
# 중복 값 제거

In [90]:
data = data.drop_duplicates(['x'])

### 동별 인구수 매핑

In [92]:
dong = gpd.read_file('읍면동격자/vl_blk.shp')

In [93]:
dong.head()

,gid,lbl,val,geometry
0,11440101,02000,40.44,"POLYGON ((951328.638 1950991.976, 951330.606 1..."
1,11440102,01000,67.74,"POLYGON ((951189.291 1949696.282, 951196.556 1..."
2,11440103,01000,61.25,"POLYGON ((952116.851 1949903.586, 952127.203 1..."
3,11440104,01000,53.36,"POLYGON ((951979.679 1949190.013, 951981.036 1..."
4,11440105,02000,35.17,"POLYGON ((950412.669 1949566.192, 950459.027 1..."


법정동코드를 사용한 동 코드 매핑

In [94]:
data['d_code'] = np.nan

In [95]:
data = data.reset_index()

In [106]:
for i in range(len(data)):
    data['d_code'][i] = str(data['시군구코드'][i]) + str(data['code_y'][i])

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [107]:
data.head()

,index,code_x,populations,code_c,code_d,시군구코드,법정동코드,행정동코드,시도명,시군구명,법정동명,행정동명,적용시작일,적용만료일,code_y,x,y,name,d_code
0,0,11440555,2.042815e+07,11440,555,11440,10200,555,서울특별시,마포구,공덕동,아현동,20080107,99991231,102,37.547105,126.952160,마포구 아현동,11440102
1,3,11440565,2.929401e+07,11440,565,11440,10300,565,서울특별시,마포구,신공덕동,공덕동,20080107,99991231,103,37.544423,126.955007,마포구 공덕동,11440103
2,7,11440585,1.674282e+07,11440,585,11440,10700,585,서울특별시,마포구,마포동,도화동,20070101,99991231,107,37.540035,126.952520,마포구 도화동,11440107
3,15,11440600,1.910087e+07,11440,600,11440,11000,600,서울특별시,마포구,노고산동,대흥동,19880423,99991231,110,37.554175,126.940118,마포구 대흥동,11440110
4,18,11440610,9.313344e+06,11440,610,11440,10200,610,서울특별시,마포구,공덕동,염리동,20150723,99991231,102,37.547066,126.945649,마포구 염리동,11440102


100m 격자에 대하여 위치별 인구 매핑

In [110]:
d_data = pd.merge( dong,data, left_on = 'gid', right_on = 'd_code')

In [111]:
d_data.shape

(15, 23)

In [112]:
d_data.head()

,gid,lbl,val,geometry,index,code_x,populations,code_c,code_d,시군구코드,...,시군구명,법정동명,행정동명,적용시작일,적용만료일,code_y,x,y,name,d_code
0,11440102,01000,67.74,"POLYGON ((951189.291 1949696.282, 951196.556 1...",0,11440555,2.042815e+07,11440,555,11440,...,마포구,공덕동,아현동,20080107,99991231,102,37.547105,126.952160,마포구 아현동,11440102
1,11440102,01000,67.74,"POLYGON ((951189.291 1949696.282, 951196.556 1...",18,11440610,9.313344e+06,11440,610,11440,...,마포구,공덕동,염리동,20150723,99991231,102,37.547066,126.945649,마포구 염리동,11440102
2,11440103,01000,61.25,"POLYGON ((952116.851 1949903.586, 952127.203 1...",3,11440565,2.929401e+07,11440,565,11440,...,마포구,신공덕동,공덕동,20080107,99991231,103,37.544423,126.955007,마포구 공덕동,11440103
3,11440107,01000,42.14,"POLYGON ((950932.204 1949085.962, 950937.119 1...",7,11440585,1.674282e+07,11440,585,11440,...,마포구,마포동,도화동,20070101,99991231,107,37.540035,126.952520,마포구 도화동,11440107
4,11440110,01000,56.78,"POLYGON ((950833.025 1950729.718, 950831.561 1...",15,11440600,1.910087e+07,11440,600,11440,...,마포구,노고산동,대흥동,19880423,99991231,110,37.554175,126.940118,마포구 대흥동,11440110


In [122]:
d_data1 = d_data[['gid','lbl','val','geometry','populations']]

In [123]:
type(d_data1)

geopandas.geodataframe.GeoDataFrame

In [ ]:
d_data1.to_file('d_data.shp')